In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
import nltk
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from gensim import corpora
import pickle
import gensim
from nltk.corpus import stopwords 
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from datetime import datetime
from sklearn.model_selection import train_test_split

In [0]:
def prerprocess(dat):
    #Ubah - menjadi space
    # untuk kata '\-' dihapus dan diganti ' '
    regex = r'\-'
    dat = dat.str.replace(regex, ' ', regex=True) # hapus
    # datat = datat.str.replace(regex, ' ', regex=True) # hapus
    regex = r'[^a-zA-Z\s]' 
    #selain kata '[^a-zA-Z\s]' akan dihapus dan diganti menjadi space
    # Ubah seluruh huruf dalam data film ke dalam lower case(casefolding)
    dat = dat.str.lower() # hapus   
    # datat = datat.str.lower()                           (3)

    # Hapus seluruh simbol yang ada dan ganti dengan string kosong ('')
    dat = dat.str.replace(regex, '', regex=True) # hapus       (4)
    # datat = datat.str.replace(regex, '', regex=True) # hapus  
    # Hapus enter, ganti dengan space 
    # untuk kata '\n' dihapus dan diganti ' ' 
    regex = r'\n'
    dat = dat.str.replace(regex, ' ', regex=True) # hapus
    # datat = datat.str.replace(regex, ' ', regex=True) # hapus
    return dat
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None
def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

def lem(dat):
  for i, v in dat.iterrows():
    
    dat.at[i, 'review']=lemmatize_sentence(v['review'])
  return dat

def stop(dat):
  stop_words = set(stopwords.words('english')) 

  for i, v in dat.iterrows():
    words = word_tokenize(v['review'])
    
    # dat[i] = [w for w in words if not w in stop_words] 
      
    text=''
    for w in words: 
        if w not in stop_words:
          text=text+' '+w
    dat.at[i, 'review']=text
    # del words
  return dat
def prepare_text_for_lda(text):
    tokens = nltk.word_tokenize(text)
    return tokens